Funkcje potrzebne do liczenia dystansu edycji oraz operacji, które zostały wykonane:

In [1]:
import numpy as np
import sys

def damerau_levenshtein_distance(string1, string2):
    """
    Given two strings string1 and string2 computes the Damerau-Levenshtein
    between them.
    """
    n1 = len(string1)
    n2 = len(string2)
    return get_levenshtein_distance_matrix(string1, string2)[n1, n2]

def get_basic_operations(string1, string2):
    dist_matrix = get_levenshtein_distance_matrix(string1, string2)
    i, j = dist_matrix.shape
    i -= 1
    j -= 1
    # operations conducted on string1 in order to transform it into string2
    operations = []
    while i != -1 and j != -1:
        if i > 1 and j > 1 and string1[i-1] == string2[j-2] \
                 and string1[i-2] == string2[j-1]:
            if dist_matrix[i-2, j-2] < dist_matrix[i, j]:
                operations.insert(0, ('transpose', i - 1, i - 2))
                i -= 2
                j -= 2
                continue
        index = np.argmin([dist_matrix[i-1, j-1], \
                           dist_matrix[i, j-1], \
                           dist_matrix[i-1, j]])
        if index == 0:
            if dist_matrix[i, j] > dist_matrix[i-1, j-1]:
                operations.insert(0, ('replace', i - 1, j - 1))
            i -= 1
            j -= 1
        elif index == 1:
            operations.insert(0, ('insert', i - 1, j - 1))
            j -= 1
        elif index == 2:
            operations.insert(0, ('delete', i - 1, i - 1))
            i -= 1
    return operations

def execute_operations(operations, string1, string2):
    # initialise the path from string1 to string2 with the first phase, i.e.
    # the string1 itself
    strings = [string1]
    # get single letters from the input string
    string = list(string1)

    shift = 0
    for op in operations:
        i, j = op[1], op[2]
        if op[0] == 'delete':
            del string[i + shift]
            shift -= 1
        elif op[0] == 'insert':
            string.insert(i + shift + 1, string2[j])
            shift += 1
        elif op[0] == 'replace':
            string[i + shift] = string2[j]
        elif op[0] == 'transpose':
            string[i + shift], string[j + shift] = string[j + shift], string[i + shift]
        strings.append(''.join(string))
    return strings

def get_levenshtein_distance_matrix(string1, string2):
    """
    Given two strings computes creates a matrix that contains the Damerau-
    Levenshtein distance in the cell [n1, n2]
    """
    n1 = len(string1)
    n2 = len(string2)
    dl_matrix = np.zeros((n1 + 1, n2 + 1), dtype=int)
    # the first column: turn input (sub)string into an empty string
    for i in range(n1 + 1):
        dl_matrix[i, 0] = i
    # the first row: turn an empty string into the target (sub)string
    for j in range(n2 + 1):
        dl_matrix[0, j] = j
    # the matrix is analysed row-wise
    for i in range(n1):
        for j in range(n2):
            cost = 0 if string1[i] == string2[j] else 1

            # consider operations allowed in Levenshtein's distance
            dl_matrix[i+1, j+1] = min(dl_matrix[i, j+1] + 1, # insertion
                                      dl_matrix[i+1, j] + 1, # deletion
                                      dl_matrix[i, j] + cost) # substitution
            if i > 0 and j > 0 and string1[i] == string2[j-1] and \
            string1[i-1] == string2[j]:
                # handle transposition
                dl_matrix[i+1, j+1] = min(dl_matrix[i+1, j+1], \
                                        dl_matrix[i-1, j-1] + cost)
    return dl_matrix

<hr>

In [4]:
from sklearn.model_selection import train_test_split
import pickle

Importuję wybrane błędy:

In [5]:
with open("../pickles/chosen_errors.p", "rb") as pickle_file:
    chosen_errors = pickle.load(pickle_file)

In [6]:
chosen_errors.shape

(609572, 9)

In [17]:
chosen_errors.head()

,text_with_error,corrected_text,is_valid_sentence,error,correction,type,dist,category,file
0,"Полежан) - najwyższy szczyt w górach Pirin, gr...","Полежан) - najwyższy szczyt w górach Pirin, gr...",False,"granotowy,","granitowy,",nonword,1,pisownia,plewic.09.0138.yaml
1,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,True,kościołą,kościoła,nonword,1,znaki diakrytyczne,plewic.09.0138.yaml
2,! '... który polski błogosławiny czczony jest ...,! '... który polski błogosławiony czczony jest...,False,błogosławiny,błogosławiony,nonword,1,pisownia,plewic.09.0138.yaml
3,! '... że NASA zleciło stworzenie gry komutero...,! '... że NASA zleciło stworzenie gry komputer...,False,komuterowej?,komputerowej?,nonword,1,pisownia,plewic.09.0138.yaml
4,V Mistrzostwa Ameryki Południowej w piłce siat...,V Mistrzostwa Ameryki Południowej w piłce siat...,True,Chle,Chile,nonword,1,pisownia,plewic.09.0138.yaml


Tworzę ramkę danych, gdzie dystans edycji dla każdej korekty jest równy 1.

In [18]:
chosen_errors_unit_distance = chosen_errors.loc[chosen_errors['dist'] == '1']

In [19]:
chosen_errors_unit_distance['dist'].unique()

array(['1'], dtype=object)

In [20]:
chosen_errors_unit_distance.shape

(548953, 9)

Kolumna `basic_type_operation` wskazuje na jedną z czterech operacji koniecznych do wykonania w celu naprawienia błędu, natomiast `nums` to kolumna informująca, które znaki trzeba wykorzystać w tej operacji. Dla każdej z operaji elememnty krotki w `nums` są inaczej interpretowane:

**Insert**:

Słowo `error` zostaje rozbite na części `error[:nums[0]+1] + correction[nums[1]] + error[nums[0]:]`. Litery, które mnie interesują to ta w `error`, po której następuje wstawienie (`error[nums[0]]`) oraz ta która jest wstawiana (`correction[nums[1]]`).

**Delete**:

Ze słowa `error` należy usunąć znak `error[nums[0]]`. Litery, które mnie interesują to ta, która stoi przed usuwaną (`error[nums[0]-1]`) oraz ta usuwana (`error[nums[0]]`).

**Replace**: 

Znak `error[nums[0]]` zastępuję znakiem `correction[nums[1]]`. Są to dwie litery, które mnie interesują.

**Transposition**:

Zamieniam ze sobą miejscami znaki `error[nums[0]]` oraz `error[nums[1]]`. Są to dwie litery, które mnie interesują.

In [24]:
get_basic_operations('abc', 'abce')[0]

('insert', 2, 3)

In [30]:
chosen_errors_unit_distance_mini = chosen_errors_unit_distance.iloc[:100]

In [31]:
chosen_errors_unit_distance_mini['basic_type_operation'] = chosen_errors_unit_distance_mini[['error', 'correction']].apply(lambda x: get_basic_operations(*x)[0][0], axis = 1)
chosen_errors_unit_distance_mini['nums'] = chosen_errors_unit_distance_mini[['error', 'correction']].apply(lambda x: get_basic_operations(*x)[0][1:], axis = 1)

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Logika error model jest taka, że operacje określam na drodze: co zrobić, żeby przejść z poprawnego kandydata do niepoprawnej, rzeczywistej obserwacji:

In [22]:
chosen_errors_unit_distance['basic_type_operation'] = chosen_errors_unit_distance[['correction', 'error']].apply(lambda x: get_basic_operations(*x)[0][0], axis = 1)
chosen_errors_unit_distance['nums'] = chosen_errors_unit_distance[['correction', 'error']].apply(lambda x: get_basic_operations(*x)[0][1:], axis = 1)

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
chosen_errors_unit_distance.head()

,text_with_error,corrected_text,is_valid_sentence,error,correction,type,dist,category,file,basic_type_operation,nums
0,"Полежан) - najwyższy szczyt w górach Pirin, gr...","Полежан) - najwyższy szczyt w górach Pirin, gr...",False,"granotowy,","granitowy,",nonword,1,pisownia,plewic.09.0138.yaml,replace,"(4, 4)"
1,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,True,kościołą,kościoła,nonword,1,znaki diakrytyczne,plewic.09.0138.yaml,replace,"(7, 7)"
2,! '... który polski błogosławiny czczony jest ...,! '... który polski błogosławiony czczony jest...,False,błogosławiny,błogosławiony,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(10, 10)"
3,! '... że NASA zleciło stworzenie gry komutero...,! '... że NASA zleciło stworzenie gry komputer...,False,komuterowej?,komputerowej?,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(3, 3)"
4,V Mistrzostwa Ameryki Południowej w piłce siat...,V Mistrzostwa Ameryki Południowej w piłce siat...,True,Chle,Chile,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(2, 2)"


In [24]:
chosen_errors_unit_distance.shape

(548953, 11)

Macierze, w którym znajdują się pary liter będę informacją dla modelu na podstawie której model wybierze słowo poprawne. Z tego powodu należy dokonać podziału na dane treningowe i testowe, aby móc dokonać wiarygodnej ewaluacji.

In [25]:
chosen_errors_unit_distance['category'].value_counts(normalize=True)

pisownia                       0.542116
znaki diakrytyczne             0.340862
znaki diakrytyczne/kontekst    0.071514
fleksja/liczba                 0.026066
składnia                       0.019442
Name: category, dtype: float64

In [26]:
chosen_errors_unit_distance['type'].value_counts(normalize=True)

nonword      0.864396
realword     0.117019
unknown      0.016029
multiword    0.002556
Name: type, dtype: float64

Usuwam kategorie `unknown` oraz `multiword`:

In [37]:
delete_mask = (chosen_errors_unit_distance['type'] != 'multiword') & (chosen_errors_unit_distance['type'] != 'unknown')

In [38]:
chosen_errors_unit_distance_ready = chosen_errors_unit_distance[delete_mask]

In [43]:
chosen_errors_unit_distance['type'].value_counts()

nonword     474513
realword     64238
Name: type, dtype: int64

In [44]:
chosen_errors_unit_distance.shape

(538751, 11)

Zapisuję jako pickle:

In [41]:
with open("../pickles/error_chosen_two_categories.p", "wb") as file:
    pickle.dump(chosen_errors_unit_distance, file, protocol=pickle.HIGHEST_PROTOCOL)

Oba zestawy będą posiadały podobny rozkład `category`. Dokonuję rozkładu ze stratyfikacją. Na razie zostawiam podział 75%/25%, potem o tym poczytam. Zmienną zależną będzie `correction`. Zostawiam w ramce danych tylko następujące kolumny:

In [58]:
cols_to_keep = ['error', 'correction', 'type', 'category', 'basic_type_operation', 'nums']

In [59]:
chosen_errors_unit_distance = chosen_errors_unit_distance[cols_to_keep]

In [61]:
y = chosen_errors_unit_distance['correction']
del chosen_errors_unit_distance['correction']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(chosen_errors_unit_distance, y, train_size = 0.75, stratify=chosen_errors_unit_distance[['category', 'type']])

/home/pawel/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [66]:
X_train['category'].value_counts(normalize=True)

pisownia                       0.540107
znaki diakrytyczne             0.340241
znaki diakrytyczne/kontekst    0.072861
składnia                       0.024064
fleksja/liczba                 0.022727
Name: category, dtype: float64

In [67]:
X_test['category'].value_counts(normalize=True)

pisownia                       0.539078
znaki diakrytyczne             0.340681
znaki diakrytyczne/kontekst    0.074148
składnia                       0.024048
fleksja/liczba                 0.022044
Name: category, dtype: float64

In [72]:
X_train['type'].value_counts(normalize=True)

nonword     0.880348
realword    0.119652
Name: type, dtype: float64

In [73]:
X_test['type'].value_counts(normalize=True)

nonword     0.87976
realword    0.12024
Name: type, dtype: float64

Do ramki testowej z powrotem dołączam kolumnę `correction`:

In [68]:
X_train['correction'] = y_train

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
X_train.head()

,error,type,category,basic_type_operation,nums,correction
629,prowadza,realword,znaki diakrytyczne/kontekst,replace,"(7, 7)",prowadzą
615,nimiecką,nonword,pisownia,insert,"(1, 2)",niemiecką
649,pierwsa,nonword,pisownia,insert,"(5, 6)",pierwsza
607,Karaibskiem,nonword,pisownia,delete,"(9, 9)",Karaibskim
549,włoadzy.,nonword,pisownia,delete,"(2, 2)",władzy.


In [75]:
X_train['basic_type_operation'].value_counts()

replace      825
insert       389
delete       187
transpose     95
Name: basic_type_operation, dtype: int64

Tworzę kolumnę na podstawie wcześniej wspomnianych par liter:

In [81]:
# pamiętaj o użyciu axis = 1
def create_column_with_letters(nums, basic_type, error, correction):
    if basic_type == 'insert':
        return (error[nums[0]], correction[nums[1]])
    elif basic_type == 'delete':
        return (error[nums[0]-1], error[nums[0]])
    elif basic_type == 'replace':
        return (error[nums[0]], correction[nums[1]])
    else:
        # transposition
        return (error[nums[0]], error[nums[1]])

X_train['letter_pairs'] = X_train[['nums', 'basic_type_operation', 'error', 'correction']].apply(lambda x: create_column_with_letters(*x), axis=1)
X_train.head()

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,error,type,category,basic_type_operation,nums,correction,letter_pairs
629,prowadza,realword,znaki diakrytyczne/kontekst,replace,"(7, 7)",prowadzą,"(a, ą)"
615,nimiecką,nonword,pisownia,insert,"(1, 2)",niemiecką,"(i, e)"
649,pierwsa,nonword,pisownia,insert,"(5, 6)",pierwsza,"(s, z)"
607,Karaibskiem,nonword,pisownia,delete,"(9, 9)",Karaibskim,"(i, e)"
549,włoadzy.,nonword,pisownia,delete,"(2, 2)",władzy.,"(ł, o)"


## Przygotowanie macierzy błędów

In [4]:
with open('../pickles/error_chosen_two_categories.p', 'rb') as file:
    error_chosen_two_categories = pickle.load(file)

In [5]:
error_chosen_two_categories.head()

,text_with_error,corrected_text,is_valid_sentence,error,correction,type,dist,category,file,basic_type_operation,nums
0,"Полежан) - najwyższy szczyt w górach Pirin, gr...","Полежан) - najwyższy szczyt w górach Pirin, gr...",False,"granotowy,","granitowy,",nonword,1,pisownia,plewic.09.0138.yaml,replace,"(4, 4)"
1,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,Rozpoczyna się przy ulicy Prymasa Stefana Wysz...,True,kościołą,kościoła,nonword,1,znaki diakrytyczne,plewic.09.0138.yaml,replace,"(7, 7)"
2,! '... który polski błogosławiny czczony jest ...,! '... który polski błogosławiony czczony jest...,False,błogosławiny,błogosławiony,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(10, 10)"
3,! '... że NASA zleciło stworzenie gry komutero...,! '... że NASA zleciło stworzenie gry komputer...,False,komuterowej?,komputerowej?,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(3, 3)"
4,V Mistrzostwa Ameryki Południowej w piłce siat...,V Mistrzostwa Ameryki Południowej w piłce siat...,True,Chle,Chile,nonword,1,pisownia,plewic.09.0138.yaml,delete,"(2, 2)"


In [7]:
error_chosen_two_categories['dist'].unique()

array(['1'], dtype=object)

Usuwam kolumnę `correction`, która będzie zmienną zależną.

In [8]:
y = error_chosen_two_categories['correction']
del error_chosen_two_categories['correction']

Podział na dane treningowe i testowe ze stratyfikacją:

In [9]:
X_train, X_test, y_train, y_test = train_test_split(error_chosen_two_categories, y, train_size = 0.75, stratify=error_chosen_two_categories[['category', 'type']])

/home/pawel/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [19]:
test_train = [X_train, X_test, y_train, y_test]

with open('../pickles/test_train.dat', 'wb') as f:
    pickle.dump(test_train, f)

Do stworzenia macierzy liter będzie potrzbny mi zestaw treningowy z zmienną zależną:

In [12]:
X_train['correction'] = y_train

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
X_train.head()

,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,correction
119,Pod koniec tego samego roku przszło na świat d...,Pod koniec tego samego roku przyszło na świat ...,True,przszło,nonword,1,pisownia,plewic.01.0427.yaml,delete,"(3, 3)",przyszło
296,Zna powiązania Eliasza z mesjanistycznymi ocze...,Zna powiązania Eliasza z mesjanistycznymi ocze...,True,"wię,",nonword,1,znaki diakrytyczne,plewic.01.0040.yaml,replace,"(2, 2)","wie,"
441,"Na początku cala okolica byla osobną wioską, s...","Na początku cała okolica była osobną wioską, s...",True,cala,realword,1,znaki diakrytyczne/kontekst,plewic.01.0026.yaml,replace,"(2, 2)",cała
440,"Była więc to działalnośc analogiczna do tej, k...","Była więc to działalność analogiczna do tej, k...",True,działalnośc,nonword,1,znaki diakrytyczne,plewic.04.0067.yaml,replace,"(10, 10)",działalność
26,Kościół posiada nawę z dwoma podcieniami i cen...,Kościół posiada nawę z dwoma podcieniami i cen...,True,podwyzszonym,nonword,1,znaki diakrytyczne,plewic.01.0036.yaml,replace,"(5, 5)",podwyższonym
